## 01 Реализация эпсилон-жадного алгоритма

Сначала мы определим класс объектов, который представляет алгоритм epsilon-Greedy в том виде, в котором он будет использоваться в реальной жизни. Этот класс будет содержать следующую информацию:

- **epsilon**

    - Это число с плавающей точкой, которое определяет частоту, с которой мы должны исследовать одно из доступных оружий. Если мы установим epsilon = 0,1, то мы будем исследовать доступные руки в 10 % случаев.

- **counts**

    - Вектор целых чисел длины N, который говорит нам, сколько раз мы сыграли на каждой из N рук, доступных нам в текущей задаче с бандитом. Если есть две руки, рука 1 и рука 2, которые были сыграны дважды, то мы зададим counts = [2, 2].

- **values**

    - Вектор чисел с плавающей точкой, определяющий среднее количество вознаграждения, которое мы получили, играя каждым из N доступных нам оружий. Если рука 1 дала нам 1 единицу награды в одной игре и 0 в другой, а рука 2 дала нам 0 единиц награды в обеих играх, то мы зададим значения = [0.5, 0.0].

Собрав все эти части вместе в правильное определение класса, мы получим следующий фрагмент кода:

```python
class EpsilonGreedy():
    def __init__(self, epsilon, counts, values):
        self.epsilon = epsilon
        self.counts = counts
        self.values = values
        return
```

Поскольку поведение алгоритма epsilon-Greedy очень сильно зависит от настроек счетчиков и значений, мы также предоставляем явные методы инициализации, которые позволяют вам вернуть эти переменные в состояние, соответствующее чистому:

```python
def initialize(self, n_arms):
    self.counts = [0 for col in range(n_arms)]
    self.values = [0.0 for col in range(n_arms)]
    return
```

Теперь, когда у нас есть класс, представляющий всю информацию, которую алгоритм epsilon-Greedy должен хранить о каждой из рук, нам нужно определить два типа поведения, которые должен обеспечивать любой алгоритм для решения проблемы многорукого бандита:

- **select_arm**
    - Каждый раз, когда нам нужно сделать выбор, какую руку тянуть, мы хотим иметь возможность просто вызвать наш любимый алгоритм, чтобы он сообщил нам порядковый номер руки, которую мы должны тянуть. Все алгоритмы бандитов будут реализовывать метод select_arm, который вызывается без каких-либо аргументов и возвращает индекс следующей руки, которую нужно потянуть.
    
- **update** 
    - После того как мы потянем за руку, мы получим от нашей системы сигнал о вознаграждении. Мы хотим обновить информацию о качестве руки, которую мы только что выбрали, предоставив информацию о вознаграждении. Все алгоритмы бандитов делают это, предоставляя функцию обновления, которая принимает в качестве аргументов (1) объект алгоритма, (2) числовой индекс последней выбранной руки и (3) вознаграждение, полученное при выборе этой руки. Метод обновления получает эту информацию и вносит соответствующие изменения в оценку алгоритмом всех рук.

Помня об общей структуре поведения, которую мы ожидаем от бандитского алгоритма, давайте рассмотрим конкретное определение этих двух функций для алгоритма epsilon-Greedy. Сначала реализуем select_arm:

```python
def ind_max(x):
    m = max(x)
    return x.index(m)

def select_arm(self):
    if random.random() > self.epsilon:
        return ind_max(self.values)
    else:
        return random.randrange(len(self.values))
```

Как видите, алгоритм epsilon-Greedy обрабатывает выбор руки в двух частях: (1) мы бросаем монету, чтобы проверить, выберем ли мы лучшую руку, о которой знаем, а затем (2) если монета выпадает решкой, мы выбираем руку совершенно случайно. В Python мы реализовали это, проверив, не превышает ли случайно сгенерированное число значение эпсилон. Если да, то наш алгоритм выбирает руку, чье кэшированное значение в соответствии с полем значений больше; в противном случае он выбирает руку случайным образом.

Эти несколько строк кода полностью описывают решение задачи "Бандит" алгоритмом epsilon-Greedy: он исследует некоторый процент времени, а в остальное время выбирает руку, которую считает лучшей. Но чтобы понять, какую руку наш алгоритм epsilon-Greedy считает лучшей, нам нужно определить функцию обновления. Давайте сделаем это сейчас, а затем объясним, почему выбранная нами процедура является разумной:

```python
def update(self, chosen_arm, reward):
    # обновили каунты
    self.counts[chosen_arm] = self.counts[chosen_arm] + 1
    n = self.counts[chosen_arm]
    
    # обновили значения
    value = self.values[chosen_arm]
    new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
    self.values[chosen_arm] = new_value
    return
```

- Взглянув на этот код, мы увидим, что функция обновления сначала увеличивает поле counts, в котором записано количество раз, когда мы играли на каждой из рук для этой задачи с бандитами, чтобы отразить выбранную руку. 
- Если это наш первый опыт работы с выбранной рукой, то мы устанавливаем оценочное значение непосредственно на вознаграждение, которое мы только что получили от игры на этой руке. 
- Если мы уже играли с этой рукой в прошлом, мы обновляем оценочную стоимость выбранной руки, чтобы она была средневзвешенным значением ранее оцененной стоимости и награды, которую мы только что получили. 
- Это взвешивание важно, потому что оно означает, что отдельные наблюдения значат для алгоритма все меньше и меньше, когда у нас уже есть большой опыт работы с каким-либо конкретным вариантом. Выбранное нами взвешивание призвано гарантировать, что расчетная стоимость будет точно равна среднему значению вознаграждения, полученного от каждой руки.

**Уточнение к new_value**: чтобы из прошлого среднего получить сумму, то я должен это среднее домножить на (n-1). Таким образом, можно просто к этой сумме добавить новое значение и получить новую сумму.

**Замечание**: Мы говорим о том, как вычислять средние значения в режиме онлайн, потому что на практике поведение бандитских алгоритмов во многом определяется этим правилом вычисления средних значений. Далее мы поговорим об альтернативных схемах взвешивания, которые вы можете использовать вместо вычисления средних. Эти альтернативные схемы взвешивания очень важны, когда руки, на которых вы играете, могут менять свое вознаграждение с течением времени.

Но сейчас давайте сосредоточимся на том, что мы сделали до сих пор. Определение класса EpsilonGreedy и определения методов select_arm и up date для этого класса полностью определяют нашу реализацию алгоритма epsilon-Greedy. Чтобы вы могли опробовать алгоритм до его развертывания, мы потратим время на создание типа фреймворка для юнит-тестирования бандитских алгоритмов.

In [16]:
import random

def ind_max(x):
    m = max(x)
    return x.index(m)

class EpsilonGreedy:
    def __init__(self, epsilon, counts, values):
        self.epsilon = epsilon
        self.counts = counts
        self.values = values

    def initialize(self, n_arms):
        self.counts = [0 for col in range(n_arms)]
        self.values = [0.0 for col in range(n_arms)]

    def select_arm(self):
        if random.random() > self.epsilon:
            return ind_max(self.values)
        else:
            return random.randrange(len(self.values))

    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] = self.counts[chosen_arm] + 1
        n = self.counts[chosen_arm]

        value = self.values[chosen_arm]
        new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
        self.values[chosen_arm] = new_value

Если мы хотим настроить алгоритм на выбор абсолютно случайным образом (аналог A/B теста):

In [19]:
algo = EpsilonGreedy(epsilon=1.0, counts=[], values=[])
algo.initialize(n_arms=2)

Настройка на максимизацию прибыли:

In [20]:
algo.epsilon = 0.0

Эти три строки полностью описывают подходы, которые буквально являются двумя самыми крайними способами, которыми можно настроить алгоритм epsilon-Greedy.

В контексте задачи выбора лучшего логотипа на сайте:

- epsilon = 1,0
    - Если мы зададим epsilon = 1,0, алгоритм всегда будет выбирать между разными руками совершенно случайным образом. В результате вы получите много данных об обоих цветовых логотипах и очень чистые данные, потому что все руки будут иметь равное количество данных и не будет никаких скрытых помех, которые мешают понять, почему вы получили те результаты, которые получили. Если вы традиционно образованный ученый, такой тип случайных экспериментов покажется вам отличным подходом. Но если вы занимаетесь бизнесом, то можете потерять много денег, потому что это означает, что вы с такой же вероятностью попробуете плохую идею, как и хорошую.
    - Или, говоря иначе: если вы управляете бизнесом, вам не стоит накапливать много данных о плохих вариантах. Вы должны собирать данные только о тех вариантах, которые, возможно, стоит реализовать. В этом и заключалась критика: установив epsilon = 1,0, вы тратите ресурсы на сбор данных о плохих вариантах. Возвращаясь к нашему примеру с логотипами, если вы выбираете один из двух цветов совершенно наугад, вы решили, что будете показывать клиентам свой худший логотип ровно столько же раз, сколько и лучший.
    
- epsilon = 0,0
    - Если вы в конце концов установите epsilon = 0,0, то действительно перестанете тратить время на плохие варианты. Но вы больше никогда не сможете узнать о новых вариантах. Если мир меняется, а вы не предоставляете своей компании никакого механизма для изучения изменений в мире, ваша компания останется позади.
    
К счастью, нет никаких причин действовать в одной из этих двух крайностей. Вместо того чтобы переходить от одного периода совершенно случайных экспериментов к другому периоду абсолютно жадного выбора так называемого лучшего варианта, алгоритм epsilon-Greedy позволяет вам действовать более постепенно. Например, вы можете установить epsilon = 0,1 и оставить алгоритм работать вечно. Во многих ситуациях это может быть лучшим вариантом.

**Но у этого подхода есть и слабые стороны.**

Первая слабость заключается в том, что по мере того, как вы все больше убеждаетесь в том, какой из двух дизайнов логотипа лучше, тенденция исследовать худший дизайн целых 5 % времени будет становиться все более расточительной. На жаргоне бандитских алгоритмов, вы будете слишком много исследовать. И есть еще одна проблема с фиксированным правилом исследования в 10 %: в начале экспериментов вы будете выбирать варианты, о которых мало что знаете, гораздо реже, чем вам хотелось бы, потому что вы пробуете новые варианты только в 10 % случаев.

Таким образом, существует множество способов улучшить алгоритм epsilon-Greedy.